In [1]:
import numpy as np
import os
import sys
from iautils import cascade
import pandas as pd
from datetime import datetime
from astropy.coordinates import SkyCoord

/opt/pysetup/.venv/lib/python3.11/site-packages/chime_frb_api/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution as _get_distribution


In [2]:
sys.path.append("/arc/home/mseth/.local/lib/python3.11/site-packages")
import memory_profiler
%load_ext memory_profiler

In [3]:
sys.path.insert(0, os.path.abspath('beam-model'))
from beam_model import utils

In [4]:
source_name = "TAU_A"
coords = SkyCoord.from_name(source_name)
source_ra = coords.ra.deg
source_dec = coords.dec.deg

For crab files, pull out all the necessary data and save to a .npz file

In [9]:
files = np.load("/arc/projects/chime_frb/mseth/Crab_Filepaths_1.npz")
crab_norescaled_filepaths = files['filepath']

In [10]:
crab_norescaled_filepaths

array(['/arc/projects/chime_frb/adamdong/for_meena/cascade_313762773_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_286399795_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_380484777_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_379943748_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_290197837_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_379943742_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_293591078_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_292900220_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_313762443_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_376722682_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_380477137_norescale.npz',
       '/arc/projects/chime_frb/adamdong/for_meena/cascade_403716

In [11]:
dms = []
max_tidxs = []
max_timestamps = []
spectra = []
beam_ids = []
has_at_peak = []
y_at_peak = []
good_filepaths = []

for filepath in crab_norescaled_filepaths:
    try:
        cascade_obj = cascade.load_cascade_from_file(filepath) 
        cascade_obj.dm=56.77 
        dm = cascade_obj.best_snr_dm
        
        if 55 <= dm <= 57:
            ts = np.nansum(cascade_obj.beams[cascade_obj._max_beam_idx].intensity, axis=0)
            max_tidx = np.argmax(ts)
            max_timestamp = cascade_obj.event_time
            spectrum_at_peak = cascade_obj.beams[cascade_obj._max_beam_idx].intensity[:, max_tidx]
            beam_id = cascade_obj.beams[cascade_obj._max_beam_idx].beam_no
            
            x, y = utils.get_position_from_equatorial(source_ra, source_dec, max_timestamp)
           
            dms.append(dm)
            max_tidxs.append(max_tidx)
            max_timestamps.append(max_timestamp)
            beam_ids.append(beam_id)
            has_at_peak.append(x)
            y_at_peak.append(y)
            spectra.append(spectrum_at_peak)
            good_filepaths.append(filepath)
            
            print(f"{filepath} good")
            del cascade_obj
        else:
            print(f"{filepath} bad")
            del cascade_obj
            continue
        
    except Exception as e:
        print(e)
        del cascade_obj
        continue

Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
Preprocessing...
using L1 weights
De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..



/opt/pysetup/.venv/lib/python3.11/site-packages/iautils/spectra.py:435: RuntimeWarning: All-NaN slice encountered
  pad = np.nanmedian(self.intensity, axis=1)


De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as reference..
Padding shifted channels using median value..

De-dispersing to 56.77 pc cm-3..
Using 400.1953125 MHz as refere

In [13]:
np.savez("/arc/projects/chime_frb/mseth/Crab_data_56_a.npz", DM=dms, MAX_TIDX=max_tidxs, MAX_TIMESTAMP=max_timestamps, BEAM_ID=beam_ids, 
         HA=has_at_peak, Y=y_at_peak)

In [12]:
has_at_peak

[-64.19192139125713,
 -6.1687566900926925,
 -54.37156709722279,
 10.480452493899772,
 -7.290458831440718,
 10.380391426176818,
 -48.230644066423125,
 -73.31224275298855,
 -69.1476418317217,
 -62.87297788472833,
 -58.04683949722857,
 -6.204015796196793,
 -73.093487654791,
 -56.94185389351747,
 -7.025815557418466,
 -72.06449114735146,
 10.506528796516045,
 -72.86086533263963,
 -66.36967030171724]

In [14]:
spectra_at_peak = np.vstack(spectra)
np.savez("/arc/projects/chime_frb/mseth/Crab_spectra_56_a.npz", SPECTRUM=spectra_at_peak)